In [2]:
%matplotlib inline

### Aprendiendo sobre los efectos de la interferencia entre cúbits

Uno de los beneficios de la computación cuántica es su capacidad para entrelazar estos principios de tal manera que, al explicar uno, se puede describir fácilmente el otro. Hicimos esto anteriormente con respecto a la interferencia. Vamos a revisarlo y ver dónde hemos encontrado este fenómeno y su uso hasta ahora.

Primero, recuerda que, al inicio de este capítulo, describimos el experimento de la doble rendija. Allí, discutimos cómo un electrón puede actuar como una onda y como una partícula. Al actuar como una onda, vimos que el experimento ilustraba cómo los electrones viajaban y aterrizaban en ciertos puntos de la pantalla de observación. El patrón que mostraban era generalmente uno que reconocemos de la física clásica como interferencia de ondas.

El patrón tenía resultados probabilísticos a lo largo de la pantalla, como se muestra en la pantalla de observación. El centro de la pantalla tiene la mayor cantidad de electrones, y las áreas vacías a ambos lados tienen poco o ningún electrón. Esto se debe a la interferencia constructiva y destructiva de las ondas.

Hay dos tipos de interferencia, a saber:

1. **Constructiva**: La interferencia constructiva ocurre cuando los picos de dos ondas se suman y la amplitud resultante es igual a la suma positiva total de las dos ondas individuales.

2. **Destructiva**: La interferencia destructiva ocurre de manera similar a la interferencia constructiva, excepto que las amplitudes de las ondas son opuestas; al sumarlas, las ondas se cancelan entre sí.

El siguiente diagrama ilustra la interferencia constructiva y destructiva de las ondas cuando se combinan:

![image.png](./Class_01/Figures/interference.png)